In [1]:
import numpy as np
import nibabel as nib
from pathlib import Path

# Import core types
from lacuna import MaskData
from lacuna.core import VoxelMap, ParcelData, ConnectivityMatrix, ScalarMetric

# Import analyses
from lacuna.analysis import (
    FunctionalNetworkMapping,
    StructuralNetworkMapping,
    RegionalDamage,
    ParcelAggregation,
)

print("Imports successful!")

Imports successful!


## 1. Creating MaskData (formerly LesionData)

MaskData now supports multiple input methods:

In [ ]:
# Create synthetic mask data
shape = (91, 109, 91)
affine = np.diag([2.0, 2.0, 2.0, 1.0])

# Binary mask (enforced in v0.5.0)
mask_array = np.zeros(shape, dtype=np.uint8)
mask_array[40:50, 50:60, 40:50] = 1  # Lesion region

mask_img = nib.Nifti1Image(mask_array, affine)

# Method 1: From nibabel image (NEW in v0.5.0)
mask_data = MaskData(
    mask_img=mask_img,
    metadata={
        "space": "MNI152NLin6Asym",
        "resolution": 2.0,
        "subject_id": "demo_001"
    }
)

print(f"Created MaskData: {mask_data.space} @ {mask_data.resolution}mm")
print(f"Mask volume: {mask_data.get_volume_mm3():.1f} mm³")

In [ ]:
# Method 2: From file path (classic method)
# Create temporary file
import tempfile
tmp_dir = Path(tempfile.mkdtemp())
mask_path = tmp_dir / "test_mask.nii.gz"
nib.save(mask_img, mask_path)

mask_data_from_file = MaskData.from_nifti(
    lesion_path=mask_path,
    metadata={
        "space": "MNI152NLin6Asym",
        "resolution": 2.0
    }
)

print(f"Loaded MaskData from file: {mask_path.name}")

## 2. Binary Mask Validation (NEW in v0.5.0)

Masks must now contain only 0 and 1 values:

In [ ]:
# This would fail - continuous values not allowed
try:
    bad_mask = np.random.rand(*shape)
    bad_mask_img = nib.Nifti1Image(bad_mask.astype(np.float32), affine)
    MaskData(
        mask_img=bad_mask_img,
        metadata={"space": "MNI152NLin6Asym", "resolution": 2.0}
    )
except ValueError as e:
    print(f"✓ Validation caught non-binary mask: {e}")

# Binarization example
continuous_mask = np.random.rand(*shape)
binary_mask = (continuous_mask > 0.5).astype(np.uint8)
print(f"\nBinarized mask: {np.sum(binary_mask)} voxels")

## 3. Functional Network Mapping

Returns unified VoxelMap containers (formerly VoxelMapResult):

In [ ]:
# Note: This requires a registered functional connectome
# For demo purposes, we'll show the API structure

fnm = FunctionalNetworkMapping(
    connectome="GSP1000",  # Example connectome
    method="correlation",
    log_level="INFO"  # NEW: Control logging output
)

print(f"Analysis: {fnm}")
print(f"\nConnectome: {fnm.connectome}")
print(f"Method: {fnm.method}")

# Run analysis (commented out - requires actual connectome)
# result = fnm.run(mask_data)
# print(f"\nResults: {list(result.results['FunctionalNetworkMapping'].keys())}")
# correlation_map = result.results['FunctionalNetworkMapping']['correlation_map_from_mask_img']
# print(f"Type: {type(correlation_map)}  # VoxelMap")

## 4. Regional Damage Analysis

Returns ParcelData containers (formerly ROIResult) with atlas-based damage percentages:

In [ ]:
# Regional damage uses bundled atlases
regional = RegionalDamage(
    parcel_names=["Schaefer100", "Schaefer200"],  # Filter specific atlases
    threshold=0.0,
    log_level="WARNING"  # Quieter logging
)

result = regional.run(mask_data)

# Access results - dict with atlas names as keys
damage_results = result.results["RegionalDamage"]
print(f"Atlases processed: {list(damage_results.keys())}")

# Each atlas returns ParcelData
for atlas_key, parcel_data in damage_results.items():
    print(f"\n{atlas_key}:")
    print(f"  Type: {type(parcel_data).__name__}")  # ParcelData
    print(f"  Regions: {len(parcel_data.get_data())}")
    
    # Get damage percentages
    damage_dict = parcel_data.get_data()
    damaged_regions = {k: v for k, v in damage_dict.items() if v > 0}
    print(f"  Damaged regions: {len(damaged_regions)}")
    
    # Show top 3 most damaged
    top_damaged = sorted(damaged_regions.items(), key=lambda x: x[1], reverse=True)[:3]
    for region, pct in top_damaged:
        print(f"    {region}: {pct:.1f}%")

## 5. Parcel Aggregation (formerly AtlasAggregation)

Aggregate VoxelMap data by atlas parcels:

In [ ]:
# First create a VoxelMap to aggregate
# For demo, create synthetic connectivity map
correlation_data = np.random.randn(*shape).astype(np.float32)
correlation_img = nib.Nifti1Image(correlation_data, affine)

correlation_map = VoxelMap(
    name="correlation_map",
    data=correlation_img,
    space="MNI152NLin6Asym",
    resolution=2.0,
    metadata={"method": "correlation", "connectome": "demo"}
)

print(f"Created VoxelMap: {correlation_map.name}")
print(f"  Space: {correlation_map.space}")
print(f"  Resolution: {correlation_map.resolution}mm")

# Add to MaskData results
mask_data.results["DemoAnalysis"] = {"correlation_map": correlation_map}

# Now aggregate by atlas
aggregation = ParcelAggregation(
    source="DemoAnalysis.correlation_map",  # Cross-analysis reference
    aggregation="mean",
    parcel_names=["Schaefer100"],
    threshold=0.0
)

agg_result = aggregation.run(mask_data)
parcel_data = agg_result.results["ParcelAggregation"]["Schaefer100_from_correlation_map"]

print(f"\nAggregated to {len(parcel_data.get_data())} parcels")
print(f"Aggregation method: {parcel_data.aggregation_method}")

# Show sample values
sample_parcels = list(parcel_data.get_data().items())[:5]
for parcel, value in sample_parcels:
    print(f"  {parcel}: {value:.3f}")

## 6. Data Container Features

All unified containers share common functionality:

In [ ]:
# Metadata storage
print(f"VoxelMap metadata: {correlation_map.metadata}")

# Summary strings
print(f"\nVoxelMap summary: {correlation_map.summary()}")
print(f"ParcelData summary: {parcel_data.summary()}")

# Data access
print(f"\nVoxelMap data type: {type(correlation_map.get_data())}")
print(f"ParcelData data type: {type(parcel_data.get_data())}")

# Container type identification
print(f"\nContainer types:")
print(f"  VoxelMap: {correlation_map.data_type}")
print(f"  ParcelData: {parcel_data.data_type}")

## 7. ScalarMetric for Summary Statistics

Store scalars, dictionaries, or any other data:

In [ ]:
# Scalar value
mean_corr = ScalarMetric(
    name="mean_correlation",
    data=0.42,
    metadata={"units": "Pearson r"}
)

print(f"Scalar: {mean_corr.summary()}")
print(f"Data type: {mean_corr.data_type}")  # Inferred as "scalar"

# Dictionary of stats
summary_stats = ScalarMetric(
    name="summary_statistics",
    data={
        "mean": 0.42,
        "std": 0.15,
        "min": -0.3,
        "max": 0.9,
        "n_voxels": 1000
    }
)

print(f"\nDictionary: {summary_stats.summary()}")
print(f"Data type: {summary_stats.data_type}")  # Inferred as "dictionary"

# Custom type label
custom_metric = ScalarMetric(
    name="lesion_severity",
    data="moderate",
    data_type="categorical"  # Explicit type
)

print(f"\nCustom: {custom_metric.summary()}")
print(f"Data type: {custom_metric.data_type}")  # Uses provided type

## 8. Input/Output Symmetry (US6)

Containers work as both inputs and outputs:

In [ ]:
# Create a VoxelMap as input
input_map = VoxelMap(
    name="input_connectivity",
    data=correlation_img,
    space="MNI152NLin6Asym",
    resolution=2.0
)

# Use it as input to ParcelAggregation
# (Store in MaskData first)
mask_data.results["InputTest"] = {"input_map": input_map}

aggregation = ParcelAggregation(
    source="InputTest.input_map",
    aggregation="mean",
    parcel_names=["Schaefer100"]
)

output = aggregation.run(mask_data)

# Get ParcelData output
output_parcel = output.results["ParcelAggregation"]["Schaefer100_from_input_map"]

print(f"Input container: {type(input_map).__name__}")
print(f"Output container: {type(output_parcel).__name__}")
print(f"\n✓ Same container types used for input and output!")

## 9. Logging Control

All analyses support log_level parameter:

In [ ]:
# Quiet mode - only errors
quiet_analysis = RegionalDamage(
    parcel_names=["Schaefer100"],
    log_level="ERROR"
)

print("Running with ERROR logging (should be quiet):")
result_quiet = quiet_analysis.run(mask_data)
print("Done (minimal output)\n")

# Verbose mode - detailed information
verbose_analysis = RegionalDamage(
    parcel_names=["Schaefer100"],
    log_level="DEBUG"
)

print("Running with DEBUG logging (verbose):")
result_verbose = verbose_analysis.run(mask_data)
print("Done (detailed output)")

## 10. Migration Summary

### Key Changes in v0.5.0

| Old API (v0.4.x) | New API (v0.5.0) |
|------------------|------------------|
| `from lacuna import LesionData` | `from lacuna import MaskData` |
| `LesionData(...)` | `MaskData(...)` |
| `VoxelMapResult` | `VoxelMap` |
| `ROIResult` | `ParcelData` |
| `AtlasAggregation` | `ParcelAggregation` |
| `anatomical_img` parameter | Removed (not needed) |
| Continuous masks allowed | Only binary (0/1) masks |

### New Features

- ✨ Direct nibabel input support
- ✨ Logging control via `log_level`
- ✨ Binary mask validation
- ✨ Input/output symmetry (US6)
- ✨ Unified container API

### Next Steps

- See `docs/migration_guide.md` for complete migration instructions
- Check `examples/` for more usage patterns
- Run `make test` to verify your code

In [ ]:
# Cleanup
import shutil
shutil.rmtree(tmp_dir)
print("Cleaned up temporary files")